# Solving Systems of Equations Review





In [ ]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)

We have covered the following methods to solve systems of equations:

* Gaussian Elimination
* LU Decomposition
* Gauss-Jordan Elimination
* The Inverse

For example, suppose you have the following system of equations:

$2x_1 + 2x_2 + x_3 = 8$

$4x_1 + 2x_2 + 6x_3 = 4$

$x_2 + 2x_3 = 2$

We can rewrite this system of equations as:

$Ax = b$

where $A$ is the coefficient matrix, $x$ is a column vector of the solutions, and $b$ is a column vector of the constants on the right hand side.

The associated coefficient matrix A is:

In [ ]:
A = np.array([[2,2,1],
             [4,2,6],
              [0,1,2]],dtype=float)
print(A)

[[2. 2. 1.]
 [4. 2. 6.]
 [0. 1. 2.]]


The associated column vector of constants (for the right hand side of the system of equations) is:

In [ ]:
b = np.array([[8],
              [4],
              [2]],dtype=float)

We can try any of the methods above to solve this system! Let's test them all...

## Gaussian Elimination

For Gaussian Elimination, we first need to create an augmented matrix [A|b]:

In [ ]:
A_b = np.concatenate((A, b), axis = 1)
print('Augmented matrix:\n', A_b)

Next we perform *forward elimination* using row operations until the augmented matrix is in Echelon Form:

In [ ]:
#R1 - 2*R0 --> R1
A_b[1,:] = A_b[1,:] - 2*A_b[0,:]
print(A_b)

In [ ]:
#R2 + (1/2)*R1 --> R2
A_b[2,:] = A_b[2,:] + (1/2)*A_b[1,:]
print(A_b)

Now that our system is in Echelon Form, we rewrite our system of equations as

$2x_1 + 2x_2 + x_3 = 8$

$-2x_2 + 4x_3 = -12$

$4x_3 = -4$


The next part of Gaussian Elimination is *back substitution*:

Starting with the bottom equation we find:

$x_3 = -1$.

We can plug that into the equation above it:

$-2x_2 + 4(-1) = -12$

And now we can solve for $x_2$:

$x_2 = 4$.

Now we can plug $x_3$ and $x_2$ into the top equation,

$2x_1 + 2*(4) + (-1) = 8$

and we can find $x_1$:

$x_1 = \frac{1}{2}$.

We have solved our system of equations!


$x = \begin{bmatrix}
\frac{1}{2} \\
4\\
-1\\
\end{bmatrix}$.



We can check this using code to make sure that $A$ matrix multiplied by our solutions for $x$ really does equal $b$:

In [ ]:
x = np.array([[0.5],
              [4],
              [-1]])
print(A@x)
print(b)

Or like this by using the computer to do the algebra for us:

In [ ]:
x3 = A_b[2,3]/A_b[2,2]
x2 = (A_b[1,3]-A_b[1,2]*x3)/A_b[1,1]
x1 = (A_b[0,3]-A_b[0,1]*x2-A_b[0,2]*x3)/A_b[0,0]
print(x1,x2,x3)

## LU Decomposition

Let's try another method - LU Decomposition!

LU Decomposition is similar to Gaussian Elimination but we first decompose the coefficient matrix A into two matrices that are multiplied together: LU. The L matrix is a *unit lower triangular matrix*, and the U matrix is an *upper triangular matrix*.

We can only use this method if NO ROW SWITCHES are required to reach echelon form. We are good, since we didn't have to use that move above. Phew.

We perform this decomposition using *elementary matrices* that correspond to the same row operations that we used above in the Guassian Elimination.

So our first row operation is:

R1 - 2*R0 --> R1

The *Elementary Matrix* is similar to the *Identity Matrix* but is MODIFIED to perform a row operation.

The row that is MODIFIED is the row that we want our operation to change.

The *row* that we want to change is R1, so that is the *row that will look different from the identity matrix*.

We want to modify R1 by adding (-2) * R0 to it, so the *column* of the identity matrix that we change is the 0 column since that is associated with R0.

In [ ]:
A = np.array([[2,2,1],
              [4,2,6],
              [0,1,2]],dtype=float)
print(A)
#R1 - 2*R0 --> R1
               #C0#C1#C2
E1 = np.array([[1, 0, 0], #R0
               [-2, 1, 0], #R1
               [0, 0, 1]],dtype=float) #R2
print(E1)

[[2. 2. 1.]
 [4. 2. 6.]
 [0. 1. 2.]]
[[ 1.  0.  0.]
 [-2.  1.  0.]
 [ 0.  0.  1.]]


Let's matrix multiply our elementary matrix on the LEFT of A to see if it performs the row operations that we think it should

In [ ]:
print(E1@A)

[[ 2.  2.  1.]
 [ 0. -2.  4.]
 [ 0.  1.  2.]]


Looks great!

Now let's find the elementary matrix that represents our next row operation:

R2 + (1/2)*R1 --> R2

The *row* that we want to change is R2, so that is the *row that will look different from the identity matrix*.

We want to modify R2 by adding (1/2) * R1, so the *column* of the identity matrix that we change is the 1 column since that is associated with R1.

In [ ]:
print(E1@A)
#R2 + (1/2)*R1 --> R2
               #C0#C1#C2
E2 = np.array([[1, 0, 0], #R0
               [0, 1, 0], #R1
               [0, 1/2, 1]],dtype = float) #R2
print(E2)

[[ 2.  2.  1.]
 [ 0. -2.  4.]
 [ 0.  1.  2.]]
[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.5 1. ]]


Let's matrix multiply our elementary matrix $E_2$ on the LEFT of $E_1A$ to see if it performs the row operations that we think it should:

In [ ]:
U = E2@E1@A
print(U)

[[ 2.  2.  1.]
 [ 0. -2.  4.]
 [ 0.  0.  4.]]


Looks great!

So now we have converted our coefficient matrix to an upper triangular matrix using elementary matrices. So we have the U we need!

$$U = E_k E_{k-1}...E_2E_1A$$

Next we need to find L using

$L =E_1^{-1}E_2^{-1}...E_{k-1}^{-1}E_k^{-1}$,

where these are the INVERSES of the elementary matrices that we used to find U, and they are applied in the REVERSE order.

The inverse matrices in this case are actually pretty simple to find. We just *switch the signs of the off-diagonal non-zero elements of the elementary matrices we already found to get the inverse elementary matrix*.

In [ ]:
#So this was E1
#R1 - 2*R0 --> R1
               #C0#C1#C2
E1 = np.array([[1, 0, 0], #R0
               [-2, 1, 0], #R1
               [0, 0, 1]],dtype = float) #R2

#Which means the inverse E1inv is
#R1 + 2*R0 -->R1
E1inv = np.array([[1, 0, 0], #R0
               [+2, 1, 0], #R1
               [0, 0, 1]],dtype = float) #R2

In [ ]:
#R2 + (1/2)*R1 --> R2
E2 = np.array([[1, 0, 0], #R0
               [0, 1, 0], #R1
               [0, 1/2, 1]],dtype = float) #R2

#R2 + (-1/2)*R1 --> R2
E2inv = np.array([[1, 0, 0], #R0
               [0, 1, 0], #R1
               [0, -1/2, 1]],dtype = float) #R2

Now we are ready to find L!

In [ ]:
L = E1inv@E2inv
print(L)

[[ 1.   0.   0. ]
 [ 2.   1.   0. ]
 [ 0.  -0.5  1. ]]


Note that L is an unit lower triangular matrix.

Let's check to make sure our decomposition works:

In [ ]:
print('A=LU')
print(A)
print(L@U)

A=LU
[[2. 2. 1.]
 [4. 2. 6.]
 [0. 1. 2.]]
[[2. 2. 1.]
 [4. 2. 6.]
 [0. 1. 2.]]


Great! LU is equivalent to our original coefficient matrix A, so we have found a good decomposition.

Now let's use this decomposition to solve our system of equations.

Remember that
$Ax=b$ and since $A = LU$, this becomes
$LUx=b.$

If we let $Ux=y$, then we can write this as

$$Ly=b$$

and we can use *forward substituion* to solve for $y$. Let's do this.

In [ ]:
print(L)
print(b)

[[ 1.   0.   0. ]
 [ 2.   1.   0. ]
 [ 0.  -0.5  1. ]]
[[8.]
 [4.]
 [2.]]


Our system of equations of y is:

$y_1=8$

$2y_1 + y_2=4$

$-\frac{1}{2}y_2 + y_3 = 2$

So we plug $y_1$ into the equation below:

$2(8) + y_2=4$

and we get $y_2 = -12$

Then we can plug $y_1$ and $y_2$ into the bottom equation:

$0(8) + -\frac{1}{2}(-12) + y_3 = 2$

and we get $y_3 = -4$

so
$y = \begin{bmatrix}
8 \\
-12\\
-4 \\
\end{bmatrix}$.


In [ ]:
y = np.array([[8],
             [-12],
              [-4]])

We can check to make sure that $L$ matrix multiplied by $y$ really does equal $b$:

In [ ]:
print('Ly=b')
print(L@y)
print(b)

Ly=b
[[8.]
 [4.]
 [2.]]
[[8.]
 [4.]
 [2.]]


We could also do the foward subsitution using code:

In [ ]:
y1 = b[0,0]/L[0,0]
y2 = (b[1,0]-L[1,0]*y1)/L[1,1]
y3 = (b[2,0]-L[2,1]*y2-L[2,0]*y1)/L[2,2]
y = np.array([[y1],[y2],[y3]])
print(y)

[[  8.]
 [-12.]
 [ -4.]]


Looks good!

Now we can use $Ux = y$ to solve for $x$. :

In [ ]:
print(U)
print(y)

[[ 2.  2.  1.]
 [ 0. -2.  4.]
 [ 0.  0.  4.]]
[[  8.]
 [-12.]
 [ -4.]]


So our new system of equations is:

$2x_1 + 2x_2 + x_3 = 8$

$-2x_2 + 4x_3 = -12$

$4x_3 = -4$

And we can solve this using *back substitution*.

Starting with the bottom row:

$x_3 = -1$

Then we plug that into the equation above so,

$-2x_2 + 4(-1) = -12$

and we find

$x_2 = 4$.

Now we can plug in $x_3$ and $x_2$ into our top equation:

$2x_1 + 2(4) + (-1) = 8$

Solving for $x_1$ we get:

$x_1= \frac{1}{2}$

Thankfully this is the same solution that we found with Gaussian Elimination!

$x = \begin{bmatrix}
\frac{1}{2} \\
4\\
-1\\
\end{bmatrix}$.




We can check it this way:

In [ ]:
x = np.array([[0.5],
              [4],
              [-1]])
print(U@x)
print(y)

We could also do this backward substitution using code:

In [ ]:
x3 = y[2,0]/U[2,2]
x2 = (y[1,0]-U[1,2]*x3)/U[1,1]
x1 = (y[0,0]-U[0,1]*x2-U[0,2]*x3)/U[0,0]
x = np.array([[x1],[x2],[x3]])
print(x)

## Gauss-Jordan Elimination

Gauss-Jordan Elimination starts out doing forward elimination just like Gaussian Elimination, but uses more row operations to get to *REDUCED* ECHELON FORM.

Just like Guassian Elimination we start with the Augmented Matrix [A|b]

In [ ]:
A_b = np.concatenate((A, b), axis = 1)
print('Augmented matrix:\n', A_b)

We use the same row operations to get to echelon form:

In [ ]:
#R1 - 2*R0 --> R1
A_b[1,:] = A_b[1,:] - 2*A_b[0,:]
print(A_b)

In [ ]:
#R2 + (1/2)*R1 --> R2
A_b[2,:] = A_b[2,:] + (1/2)*A_b[1,:]
print(A_b)

This is where we left off with Gaussian Elimination, but we continue on with more row operations.

Next we use row operations to make the diagonal entries 1.

In [ ]:
#(1/2)*R0-->R0
A_b[0,:] = (1/2)*A_b[0,:]
print(A_b)

In [ ]:
#(-1/2)*R1-->R1
A_b[1,:] = (-1/2)*A_b[1,:]
print(A_b)

In [ ]:
#(1/4)*R2-->R2
A_b[2,:] = (1/4)*A_b[2,:]
print(A_b)

Lastly we make all the entries above the pivots zero, starting with the bottom-right and moving upwards and then left.

In [ ]:
#R1 + 2*R2 --> R1
A_b[1,:] = A_b[1,:] + 2*A_b[2,:]
print(A_b)

In [ ]:
#R0 - (1/2)*R2 --> R0
A_b[0,:] = A_b[0,:] + (-1/2)*A_b[2,:]
print(A_b)

In [ ]:
#R0 - R1 --> R0
A_b[0,:] = A_b[0,:] - A_b[1,:]
print(A_b)

Rewriting our system of equations we have:

$x_1 = 1/2$

$x_2 = 4$

$x_3 = -1$

Thankfully this is the same solution that we found with Gaussian Elimination and LU Decomposition!

$x = \begin{bmatrix}
\frac{1}{2} \\
4\\
-1\\
\end{bmatrix}$.

We can also check this using the Matrix routine from the sympy package:

In [ ]:
from sympy import Matrix
A_b_matrix =  Matrix(A_b)

print(f'Reduced row echelon form of [A|b]=')
print(A_b_matrix.rref(),'\n')

This shows us the augmented matrix in reduced row echelon form, followed by the index of each pivot column (In this case: (0,1,2)).

This matches. Great!

## The Inverse

We find the inverse of a matrix by starting with a different augmented matrix, $[A|I]$, where $I$ is the identity matrix that is the same size as A. (A must be a square matrix for this to work.)

In [ ]:
print("A=")
print(A)
n,m = np.shape(A)
print("I=")
I = np.identity(n)
print(I)
A_I = np.concatenate((A, I), axis = 1)
print('Augmented matrix [A|I]:\n', A_I)

Next we perform row operations on the augmented matrix until the left side of the matrix turns into the identity matrix. (This is only possible if the matrix is "invertible").

These will be the same row operations that we used for Guass-Jordan Elimination.

In [ ]:
#R1 - 2*R0 --> R1
A_I[1,:] = A_I[1,:] - 2*A_I[0,:]
print(A_I)

In [ ]:
#R2 + (1/2)*R1 --> R2
A_I[2,:] = A_I[2,:] + (1/2)*A_I[1,:]
print(A_I)

In [ ]:
#(1/2)*R0-->R0
A_I[0,:] = (1/2)*A_I[0,:]
print(A_I)

In [ ]:
#(-1/2)*R1-->R1
A_I[1,:] = (-1/2)*A_I[1,:]
print(A_I)

In [ ]:
#(1/4)*R2-->R2
A_I[2,:] = (1/4)*A_I[2,:]
print(A_I)

In [ ]:
#R1 + 2*R2 --> R1
A_I[1,:] = A_I[1,:] + 2*A_I[2,:]
print(A_I)

In [ ]:
#R0 - (1/2)*R2 --> R0
A_I[0,:] = A_I[0,:] + (-1/2)*A_I[2,:]
print(A_I)

In [ ]:
#R0 - R1 --> R0
A_I[0,:] = A_I[0,:] - A_I[1,:]
print(A_I)

Now the right hand side of our matrix is the inverse of A!

$A^{-1} = \begin{bmatrix}
\frac{1}{8} & \frac{3}{16} & -\frac{5}{8} \\
\frac{1}{2} & -\frac{1}{4} &\frac{1}{2}\\
-\frac{1}{4} & \frac{1}{8} & \frac{1}{4}\\
\end{bmatrix}$

In [ ]:
Ainv = A_I[:,3:]
print(Ainv)

By definition, the following should be true for the inverse:

$A A^{-1} = I$.

$A^{-1} A  = I$.

Let's check this to make sure we calculated the inverse correctly.

In [ ]:
print(A@Ainv)
print(Ainv@A)

Looks good!

Now let's use this inverse to solve our system of equations.

Remember that we started out with
$Ax = b$

If we matrix multiply on the left by the inverse for both sides:

$A^{-1}Ax = A^{-1}b$

which becomes:

$x = A^{-1} b$.

So this tells us that if we want to find $x$, we simply have to multiply the inverse we found with $b$:



In [ ]:
x = Ainv@b
print(x)

It worked!

This solution matches the solution we found using all the other methods above.

We can also check this using the inv function from the np.linalg package:

In [ ]:
x = np.linalg.inv(A)@b
print(x)

Looks good!